In [109]:
#Imports
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [110]:
#Network parameters
batch_size = 64
img_height = 224
img_width = 224

In [111]:
test_folder = '../images/uk_us/test'
preprocessor = keras.applications.resnet50.preprocess_input
test_data_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocessor)
test_set = test_data_generator.flow_from_directory(test_folder, shuffle=True, class_mode='binary',target_size=(img_height,img_width))

Found 374 images belonging to 2 classes.


In [112]:
#Get pretrained model
resnet_model = tf.keras.applications.ResNet50(input_shape=(img_width, img_height, 3), include_top=False, pooling='avg', weights='imagenet')
resnet_model.trainable = False
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [113]:
#add classfier
inputs = keras.Input(shape=(img_height,img_width, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = resnet_model(inputs, training=False)
x = keras.layers.Dense(1, name='logits')(x)
outputs = keras.layers.Activation('sigmoid',name='sigmoid_out')(x)
model = keras.Model(inputs, outputs)

In [114]:
model.load_weights('./model_checkpoints/uk_us/seq_location_classifier')

In [115]:
def get_original_image(x):
    x += 1
    x *= 127.5
    x /= 255.
    x = x[..., ::-1]
    x = np.interp(x, (x.min(), x.max()), (0, 1))
    return x

In [116]:
images, labels = test_set.next()

In [117]:
labels

array([1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
       1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1.],
      dtype=float32)

In [118]:
sample_imgs = []
for i in range(2):
    valid = images[labels == i]
    pick = np.random.choice(range(len(valid)))
    sample_imgs.append(valid[pick])

sample_imgs = np.array(sample_imgs)

In [ ]:
last_conv = activations[5]
gradients = [tf.gradients(logits[:,i,None], last_conv)[0] for i in range(10)]
partial_lin = [tf.nn.avg_pool(gradient, [1, 10, 10, 1], [1, 1, 1, 1], 'VALID') for gradient in gradients]
grad_cams = [tf.nn.relu(tf.reduce_sum(last_conv * partial_lin[i], axis=3, keep_dims=True)) for i in range(10)]
resized_grad_cams = [tf.image.resize_bilinear(grad_cams[i], [40,40], align_corners=True) for i in range(10)]

hmaps = np.reshape([sess.run(resized_grad_cams[i], feed_dict={X: sample_imgs[i][None]}) for i in range(10)], [10, 40, 40])

In [119]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
logits (Dense)               (None, 1)                 2049      
_________________________________________________________________
sigmoid_out (Activation)     (None, 1)                 0         
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________


In [120]:
model.get_layer('conv5_block3_out').name

ValueError: No such layer: conv5_block3_out.

In [43]:
#layers
last_conv_layer = (model.layers[0]).layers[-2]
logits_layer = model.layers[-1]
last_conv_layer.name, logits_layer.name

('conv5_block3_out', 'logits')

In [74]:
resnet_model.layers[-2].name, model.layers[1].name

('conv5_block3_out', 'logits')

In [80]:
hmaps = []
for sample_image in sample_imgs:
    sample_img = tf.convert_to_tensor(sample_image, dtype=tf.float32)
    with tf.GradientTape() as g:
        g.watch(resnet_model.layers[-2].variables)
        
        with tf.GradientTape() as gg:
            gg.watch(model.layers[0].variables)
            
            logits = model(sample_img[np.newaxis,...])
            
        mod_gradients = gg.gradient(logits, model.layers[0].variables) #convert to binary cross entropy
        print(type(logits))
            
        gradients = g.gradient(mod_gradients, resnet_model.layers[-2].variables)
        print(len(gradients))
        avgpool = keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1,1), padding='valid')
        
        partial_lin = avgpool(gradients)
        reduced = tf.reduce_sum(last_conv_layer.output*partial_lin,axis=3, keep_dims=True)
        grad_cam = keras.layers.ReLU()(reduced)
        resized_grad_cam = tf.image.resize_bilinear(grad_cam, [224,224,3], align_corners=True)
        hmaps.append(resized_grad_cam)

<class 'tensorflow.python.framework.ops.EagerTensor'>


TypeError: Cannot convert value None to a TensorFlow DType.

In [83]:
hmaps = []
for sample_image in sample_imgs:
    sample_img = tf.convert_to_tensor(sample_image, dtype=tf.float32)
    with tf.GradientTape() as g:
        gradModel = keras.Model(
            inputs=[model.inputs],
            outputs=[model.get_layer(layer_name).output, model.get_layer('logits').output]
        )
        g.watch(resnet_model.layers[-2].variables)
        logits = model(sample_img[np.newaxis,...])
    gradients = g.gradient(logits, resnet_model.layers[-2].variables) #convert to binary cross entropy

    avgpool = keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1,1), padding='valid')

    partial_lin = avgpool(gradients)
    reduced = tf.reduce_sum(last_conv_layer.output*partial_lin, axis=3, keep_dims=True)
    grad_cam = keras.layers.ReLU()(reduced)
    resized_grad_cam = tf.image.resize_bilinear(grad_cam, [224,224,3], align_corners=True)
    hmaps.append(resized_grad_cam)

ValueError: Layer average_pooling2d_8 expects 1 input(s), but it received 0 input tensors. Inputs received: []

In [106]:
def grad_cam(model, image, category_index, layer_name='conv5_block3_out'):

    gradModel = keras.Model(
            inputs=[model.inputs],
            outputs=[model.get_layer(layer_name).output, model.get_layer('logits').output]
    )

    with tf.GradientTape() as tape:

        inputs = tf.cast(image, tf.float32)
        (convOutputs, predictions) = gradModel(inputs)
        

    grads = tape.gradient(predictions, convOutputs)


    castConvOutputs = tf.cast(convOutputs > 0, "float32")
    castGrads = tf.cast(grads > 0, "float32")
    guidedGrads = castConvOutputs * castGrads * grads


    convOutputs = convOutputs[0]
    guidedGrads = guidedGrads[0]

    weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)


    H, W = image.shape[1], image.shape[2]
    cam = np.maximum(cam, 0)  # ReLU so we only get positive importance
    cam = cv2.resize(cam, (W, H), cv2.INTER_NEAREST)
    cam = cam / cam.max()

    return cam

In [108]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
logits (Dense)               (None, 1)                 2049      
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________


In [107]:
hmaps = []
for i,sample_image in enumerate(sample_imgs):
    cam = grad_cam(model, sample_image, i)
    hmaps.append(cam)

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'") at layer "conv1_pad". The following previous layers were accessed without issue: []

In [104]:
ins=model.inputs

In [105]:
ins

[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'resnet50_input')>]

In [ ]:
im = load_image_normalize(im_path, mean, std)

print(im.shape)
cam = grad_cam(model, im, 5, 'conv5_block16_concat') # Mass is class 5

# Loads reference CAM to compare our implementation with.
reference = np.load("reference_cam.npy")
error = np.mean((cam-reference)**2)

print(f"Error from reference: {error:.4f}, should be less than 0.05")




plt.imshow(load_image(im_path, df, preprocess=False), cmap='gray')
plt.title("Original")
plt.axis('off')

plt.show()

plt.imshow(load_image(im_path, df, preprocess=False), cmap='gray')
plt.imshow(cam, cmap='magma', alpha=0.5)
plt.title("GradCAM")
plt.axis('off')
plt.show()